# Parse Actuals

Looks through the URLs to grab batting & pitching actuals and deliver those back to the user.

In [35]:
import requests
import csv
import datetime
from bs4 import BeautifulSoup

In [106]:
# static urls
season = datetime.datetime.now().year
PITCHERS_URL = "https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=al&qual=y&type=c,36,37,38,40,-1,120,121,217,-1,24,41,42,43,44,-1,117,118,119,-1,6,45,124,-1,62,122,13&season={season}&month=0&season1={season}&ind=0&team=0&rost=0&age=0&filter=&players=0&page=1_100000".format(season=season)
BATTERS_URL = "https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=al&qual=y&type=8&season={season}&month=0&season1={season}&ind=0&team=0&rost=0&age=0&filter=&players=0&page=1_10000".format(season=season)

# # request the data
pitchers_html = requests.get(PITCHERS_URL).text
batters_html = requests.get(BATTERS_URL).text

Now take the requests and parse out the relevant header information for each of the positions. This function will take one of the fangraphs pages as input and write out a CSV of that information once it's parsed.

In [107]:
def parse_array_from_fangraphs_html(input_html, out_file_name):
    """
    Take a HTML stats page from fangraphs and parse it out to a CSV file.
    """
    # parse input
    soup = BeautifulSoup(input_html, "lxml")
    table = soup.find("table", {"class": "rgMasterTable"})
    
    # get headers
    headers_html = table.find("thead").find_all("th")
    headers = []
    for header in headers_html:
        headers.append(header.text)
    print(headers)
    
    # get rows
    rows = []
    rows_html = table.find("tbody").find_all("tr")
    for row in rows_html:
        row_data = []
        for cell in row.find_all("td"):
            row_data.append(cell.text)
        rows.append(row_data)
    
    # write to CSV file
    with open(out_file_name, "w") as out_file:
        writer = csv.writer(out_file)
        writer.writerow(headers)
        writer.writerows(rows)
        


Now that we have all of the player data, I'm writing these out to a CSV file if I want to load them again later without having to run the requests to those pages once more.

In [108]:
parse_array_from_fangraphs_html(batters_html, 'batters_actuals.csv')
parse_array_from_fangraphs_html(pitchers_html, 'pitchers_actuals.csv')

['#', 'Name', 'Team', 'G', 'PA', 'HR', 'R', 'RBI', 'SB', 'BB%', 'K%', 'ISO', 'BABIP', 'AVG', 'OBP', 'SLG', 'wOBA', 'wRC+', 'BsR', 'Off', 'Def', 'WAR']
['#', 'Name', 'Team', 'K/9', 'BB/9', 'K/BB', 'HR/9', 'K%', 'BB%', 'K-BB%', 'SO', 'AVG', 'WHIP', 'BABIP', 'LOB%', 'ERA-', 'FIP-', 'xFIP-', 'ERA', 'FIP', 'E-F', 'xFIP', 'SIERA', 'IP']


Load those CSV files using read_csv() in pandas. Since some of the percentage values are stored as strings, we need to parse those into floats.

In [83]:
bdf = pd.read_csv('batters_actuals.csv')
pdf = pd.read_csv('pitchers_actuals.csv')

# create a function to parse out percentage strings to floats
def parse_pctg(value):
    return float(value.split()[0]) / 100

In [84]:
# apply that to all percentage values in the dataframes
bdf['BB%'] = bdf['BB%'].apply(lambda x: parse_pctg(x))
bdf['K%'] = bdf['K%'].apply(lambda x: parse_pctg(x))
pdf['K%'] = pdf['K%'].apply(lambda x: parse_pctg(x))
pdf['BB%'] = pdf['BB%'].apply(lambda x: parse_pctg(x))
pdf['K-BB%'] = pdf['K-BB%'].apply(lambda x: parse_pctg(x))
pdf['LOB%'] = pdf['LOB%'].apply(lambda x: parse_pctg(x))

In [110]:
with open('batters_projections.html', 'r') as bhtml:
    btxt = bhtml.read()
    dfb_proj = pd.read_html(btxt)[-1]  # read_html returns ALL tables, we just want the last one.
    dfb_proj.dropna(axis=1, inplace=True)

with open('pitchers_projections.html', 'r') as phtml:
    ptxt = phtml.read()
    dfp_proj = pd.read_html(ptxt)[-1]
    dfp_proj.dropna(axis=1, inplace=True)

In [111]:
dfp_proj.head()

,Name,W,L,SV,HLD,ERA,GS,G,IP,H,ER,HR,SO,BB,WHIP,K/9,BB/9,FIP,WAR
0,Aroldis Chapman,3,1,18,2,2.35,0,35,35.0,22,9,3,56,15,1.07,14.24,3.94,2.41,1.4
1,Andrew Miller,2,1,2,7,2.49,0,27,27.0,19,7,2,39,10,1.07,13.20,3.20,2.68,0.8
2,Craig Kimbrel,2,1,17,2,2.74,0,34,34.0,22,10,4,53,13,1.05,14.15,3.53,2.68,1.1
3,Kenley Jansen,2,1,18,3,2.86,0,35,35.0,27,11,4,46,9,1.02,11.95,2.27,2.76,1.2
4,Noah Syndergaard,6,4,0,0,3.04,14,14,83.0,72,28,7,98,19,1.09,10.55,2.08,2.76,2.5
